In [1]:
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String,DateTime,Boolean
from sqlalchemy.sql import func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

engine = create_engine('sqlite:///test.db', echo=True)
Base = declarative_base()

class Clients(Base):
    __tablename__ = "clients"

    id = Column('id', Integer, primary_key=True)
    name = Column('name', String(100), nullable=True)


class Status(Base):
    __tablename__ = "status"

    id = Column('id', Integer, primary_key=True)
    name = Column('name', String(100), nullable=True)

    
    
class Technicians(Base):
    __tablename__ = "technicians"

    id = Column('id', Integer, primary_key=True)
    name = Column('name', String(100), nullable=True)

    
class Tasks(Base):
    __tablename__ = "tasks"

    id = Column('id', Integer, primary_key=True)
    name = Column('name', String(100), nullable=True)
    service_type = Column('service_type', String(2), nullable=True)

class Specialties(Base):
    __tablename__ = "specialties"

    task_id = Column('task_id', Integer, ForeignKey('tasks.id'), primary_key=True)
    technician_id = Column('technician_id', Integer, ForeignKey('technicians.id'), primary_key=True)

    tasks = relationship(Tasks)
    technicians = relationship(Technicians)

    
class Orders(Base):
    __tablename__ = "orders"

    id = Column('id', Integer, primary_key=True)


    added_datetime = Column('added_datetime',DateTime)
    etc_datetime = Column('etc_datetime',DateTime)
    started_datetime = Column('started_datetime',DateTime, nullable=True)
    closed_datetime = Column('closed_datetime',DateTime, nullable=True)

    client_id = Column('client_id', Integer, ForeignKey('clients.id'), primary_key=True)
    task_id = Column('task_id', Integer, ForeignKey('tasks.id'), primary_key=True)
    status_id = Column('status_id', Integer, ForeignKey('status.id'), primary_key=True)
    tech_id = Column('tech_id', Integer, ForeignKey('technicians.id'), primary_key=True)

    clients = relationship(Clients)
    tasks = relationship(Tasks)
    status = relationship(Status)    
    technicians = relationship(Technicians)    

Base.metadata.create_all(engine) # create tables if not existing yet

#Session = sessionmaker(engine)

#with Session() as session:
#    session.add(some_object)
#    session.add(some_other_object)
#    session.commit()

2023-06-30 14:47:06,935 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-30 14:47:06,937 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clients")
2023-06-30 14:47:06,937 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-30 14:47:06,939 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("status")
2023-06-30 14:47:06,939 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-30 14:47:06,940 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("technicians")
2023-06-30 14:47:06,941 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-30 14:47:06,942 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tasks")
2023-06-30 14:47:06,942 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-30 14:47:06,943 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("specialties")
2023-06-30 14:47:06,943 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-30 14:47:06,944 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2023-06-30 14:47:06,945 INFO sqlalchemy.engine.Engine [raw sql

In [159]:
import pandas as pd

In [160]:
test_df = pd.read_excel("test_data.xlsx")
test_df = test_df[['service', 'client', 'task', 'added', 'etc', 'started', 'closed',
       'tech', 'Status']]
test_df.head()

,service,client,task,added,etc,started,closed,tech,Status
0,ME000,Cirrolytix,PCB development,15:04; 02/23/2023,11:00; 02/27/2023,16:02; 02/23/2023,10:37; 02/27/2023,J. Barba,Closed
1,TE001,SMC,Product safety test,09:48; 03/07/2023,14:00; 03/09/2023,08:09; 03/08/2023,13:14; 03/09/2023,R. Apostol,Closed
2,TE002,ABS-CBN,EMC test,10:17; 03/13/2023,12:00; 03/15/2023,11:20; 03/13/2023,13:12; 03/15/2023,J. Solomon,Closed
3,RD003,TIP-QC students,PCB design,10:54; 03/14/2023,12:00; 03/16/2023,12:30; 03/14/2023,14:05; 03/16/2023,B. Gawaran,Closed
4,ME004,PLM students,PCB development,11:32; 04/18/2023,15:00; 04/20/2023,15:57; 04/18/2023,10:58; 04/20/2023,M. Gorion,Closed


In [166]:

clients = test_df['client'].unique()
status = test_df['Status'].unique()
techs = test_df['tech'].unique()
tasks = test_df[['service','task']].apply(lambda x:(x['service'][0:2],x['task']),axis=1).unique()

# create list
client_list = []
client_dict = {}
for i,client in enumerate(clients):
    client_list.append(Clients(id=i, name=client))
    client_dict[client]=i

stat_list = []
stat_dict = {}
for i,stat in enumerate(status):
    stat_list.append(Status(id=i, name=stat))
    stat_dict[stat]=i

    
tech_list = []
tech_dict = {}
for i,tech in enumerate(techs):
    tech_list.append(Technicians(id=i, name=tech))
    tech_dict[tech]=i
    
task_list = []
task_dict = {}
tasks = test_df[['service','task']].apply(lambda x:(x['service'][0:2],x['task']),axis=1).unique()
for i,(a,b) in enumerate(tasks):
    task_list.append(Tasks(id=i,service_type=a,name=b))
    task_dict[b]=i

spez_list = []
spez = test_df[['task','tech']].apply(lambda x:(x['task'],x['tech']),axis=1).unique()
for i,(a,b) in enumerate(spez):
    spez_list.append(Specialties(task_id=task_dict[a],technician_id=tech_dict[b]))
    
order_list = []
for index, row in test_df[['added','etc','started','closed','client','task','Status','tech']].iterrows():
    added_datetime = datetime.strptime(row['added'],"%H:%M; %m/%d/%Y")
    etc_datetime = datetime.strptime(row['etc'],"%H:%M; %m/%d/%Y")
    started_datetime = datetime.strptime(row['started'],"%H:%M; %m/%d/%Y") if(row['started']!='-') else None
    closed_datetime = datetime.strptime(row['closed'],"%H:%M; %m/%d/%Y") if(row['closed']!='-') else None

    client_id = client_dict[row['client']]
    task_id = task_dict[row['task']]
    status_id = stat_dict[row['Status']]
    tech_id = tech_dict[row['tech']]
    order_tmp = Orders(id=index,added_datetime=added_datetime,etc_datetime=etc_datetime,started_datetime=started_datetime,closed_datetime=closed_datetime,client_id=client_id,task_id=task_id,status_id=status_id,tech_id=tech_id)
    order_list.append(order_tmp)


In [167]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session

Session = sessionmaker(bind=engine)
with Session() as session:
    [session.add(i) for i in client_list]
    [session.add(i) for i in stat_list]
    [session.add(i) for i in tech_list]
    [session.add(i) for i in task_list]
    [session.add(i) for i in spez_list]
    [session.add(i) for i in order_list]

    
    session.commit()

2023-06-30 01:02:58,522 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-30 01:02:58,524 INFO sqlalchemy.engine.Engine INSERT INTO clients (id, name) VALUES (?, ?)
2023-06-30 01:02:58,525 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ((0, 'Cirrolytix'), (1, 'SMC'), (2, 'ABS-CBN'), (3, 'TIP-QC students'), (4, 'PLM students'), (5, 'QBO incubees'), (6, 'dela Cruz, Juan C.'), (7, 'Analog Devices')  ... displaying 10 of 15 total bound parameter sets ...  (13, 'DOST-ITDI'), (14, 'Toshiba'))
2023-06-30 01:02:58,528 INFO sqlalchemy.engine.Engine INSERT INTO status (id, name) VALUES (?, ?)
2023-06-30 01:02:58,529 INFO sqlalchemy.engine.Engine [generated in 0.00131s] ((0, 'Closed'), (1, 'Ongoing'), (2, 'Waiting'))
2023-06-30 01:02:58,531 INFO sqlalchemy.engine.Engine INSERT INTO tasks (id, name, service_type) VALUES (?, ?, ?)
2023-06-30 01:02:58,532 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ((0, 'PCB development', 'ME'), (1, 'Product safety test', 'TE'), (2, 'EMC test',

In [168]:
from sqlalchemy import select

with Session() as session:
    # query for ``User`` objects
    tables = (Base.metadata.tables.keys())
    print(tables)
    
    result = session.query(Orders,Status,Tasks,Clients,Technicians).filter(Orders.status_id==Status.id).filter(Orders.task_id==Tasks.id).filter(Orders.client_id==Clients.id).filter(Orders.tech_id==Technicians.id).all()
    
    
    # list of Row objects

#for i in statement:
    #display(i)
for i in result:
    print(i)

dict_keys(['clients', 'status', 'technicians', 'tasks', 'specialties', 'orders'])
2023-06-30 01:03:01,243 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-30 01:03:01,244 INFO sqlalchemy.engine.Engine SELECT orders.id AS orders_id, orders.added_datetime AS orders_added_datetime, orders.etc_datetime AS orders_etc_datetime, orders.started_datetime AS orders_started_datetime, orders.closed_datetime AS orders_closed_datetime, orders.client_id AS orders_client_id, orders.task_id AS orders_task_id, orders.status_id AS orders_status_id, orders.tech_id AS orders_tech_id, status.id AS status_id, status.name AS status_name, tasks.id AS tasks_id, tasks.name AS tasks_name, tasks.service_type AS tasks_service_type, clients.id AS clients_id, clients.name AS clients_name, technicians.id AS technicians_id, technicians.name AS technicians_name 
FROM orders, status, tasks, clients, technicians 
WHERE orders.status_id = status.id AND orders.task_id = tasks.id AND orders.client_id = clients.id AND o

In [176]:
q = """SELECT orders.id AS orders_id, orders.added_datetime AS orders_added_datetime, orders.etc_datetime AS orders_etc_datetime, orders.started_datetime AS orders_started_datetime, orders.closed_datetime AS orders_closed_datetime, orders.client_id AS orders_client_id, orders.task_id AS orders_task_id, orders.status_id AS orders_status_id, orders.tech_id AS orders_tech_id, status.id AS status_id, status.name AS status_name, tasks.id AS tasks_id, tasks.name AS tasks_name, tasks.service_type AS tasks_service_type, clients.id AS clients_id, clients.name AS clients_name, technicians.id AS technicians_id, technicians.name AS technicians_name 
FROM orders, status, tasks, clients, technicians 
WHERE orders.status_id = status.id AND orders.task_id = tasks.id AND orders.client_id = clients.id AND orders.tech_id = technicians.id"""
df = pd.read_sql_query(q,con=engine)
df = df[['tasks_service_type','orders_id',  'tasks_name', 'orders_added_datetime', 'orders_etc_datetime',
       'orders_started_datetime', 'orders_closed_datetime',
        'clients_name',  'technicians_name', 
       'status_name', ]]

2023-06-30 01:34:47,587 INFO sqlalchemy.engine.Engine SELECT orders.id AS orders_id, orders.added_datetime AS orders_added_datetime, orders.etc_datetime AS orders_etc_datetime, orders.started_datetime AS orders_started_datetime, orders.closed_datetime AS orders_closed_datetime, orders.client_id AS orders_client_id, orders.task_id AS orders_task_id, orders.status_id AS orders_status_id, orders.tech_id AS orders_tech_id, status.id AS status_id, status.name AS status_name, tasks.id AS tasks_id, tasks.name AS tasks_name, tasks.service_type AS tasks_service_type, clients.id AS clients_id, clients.name AS clients_name, technicians.id AS technicians_id, technicians.name AS technicians_name 
FROM orders, status, tasks, clients, technicians 
WHERE orders.status_id = status.id AND orders.task_id = tasks.id AND orders.client_id = clients.id AND orders.tech_id = technicians.id
2023-06-30 01:34:47,589 INFO sqlalchemy.engine.Engine [raw sql] ()


In [179]:
df1 = df[['tasks_service_type','orders_id']]
df1 = df1.apply(lambda x:x['tasks_service_type']+str(x['orders_id']).zfill(3),axis=1)
df2 = df[['tasks_name', 'orders_added_datetime', 'orders_etc_datetime',
       'orders_started_datetime', 'orders_closed_datetime',
        'clients_name',  'technicians_name', 
       'status_name' ]]
final_df=pd.concat([df1,df2],axis=1)
final_df.columns=['order_id','tasks_name', 'orders_added_datetime', 'orders_etc_datetime',
       'orders_started_datetime', 'orders_closed_datetime',
        'clients_name',  'technicians_name', 
       'status_name']
final_df

,order_id,tasks_name,orders_added_datetime,orders_etc_datetime,orders_started_datetime,orders_closed_datetime,clients_name,technicians_name,status_name
0,ME000,PCB development,2023-02-23 15:04:00.000000,2023-02-27 11:00:00.000000,2023-02-23 16:02:00.000000,2023-02-27 10:37:00.000000,Cirrolytix,J. Barba,Closed
1,TE001,Product safety test,2023-03-07 09:48:00.000000,2023-03-09 14:00:00.000000,2023-03-08 08:09:00.000000,2023-03-09 13:14:00.000000,SMC,R. Apostol,Closed
2,TE002,EMC test,2023-03-13 10:17:00.000000,2023-03-15 12:00:00.000000,2023-03-13 11:20:00.000000,2023-03-15 13:12:00.000000,ABS-CBN,J. Solomon,Closed
3,RD003,PCB design,2023-03-14 10:54:00.000000,2023-03-16 12:00:00.000000,2023-03-14 12:30:00.000000,2023-03-16 14:05:00.000000,TIP-QC students,B. Gawaran,Closed
4,ME004,PCB development,2023-04-18 11:32:00.000000,2023-04-20 15:00:00.000000,2023-04-18 15:57:00.000000,2023-04-20 10:58:00.000000,PLM students,M. Gorion,Closed
5,ME005,Component soldering,2023-04-19 16:08:00.000000,2023-04-21 13:00:00.000000,2023-04-20 12:54:00.000000,2023-04-21 14:02:00.000000,QBO incubees,M. Gorion,Closed
6,ME006,Component soldering,2023-04-25 15:45:00.000000,2023-04-27 10:00:00.000000,2023-04-26 15:23:00.000000,2023-04-27 14:10:00.000000,"dela Cruz, Juan C.",J. Barba,Closed
7,TE007,Parametric test,2023-04-26 11:02:00.000000,2023-04-28 11:00:00.000000,2023-04-27 09:49:00.000000,2023-04-28 15:19:00.000000,Analog Devices,M. Ganzo,Closed
8,RD008,Enclosure 3D printing,2023-05-10 12:22:00.000000,2023-05-12 14:00:00.000000,2023-05-11 11:40:00.000000,2023-05-12 14:09:00.000000,WattSmart Phil Corp,R. Gumtang,Closed
9,RD009,PCB design,2023-05-10 14:10:00.000000,2023-05-12 12:00:00.000000,2023-05-11 10:29:00.000000,2023-05-12 17:20:00.000000,UP EEEI students,B. Gawaran,Closed


In [6]:
import pandas as pd

q = """select max(id) from clients"""
df = pd.read_sql_query(q,con=engine)
df.values[0][0]

2023-06-30 14:48:40,835 INFO sqlalchemy.engine.Engine select max(id) from clients
2023-06-30 14:48:40,836 INFO sqlalchemy.engine.Engine [raw sql] ()


14